# Day 9

In [375]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [376]:
with open('/Users/williamkirkland/Data/KDS/adventofcode/2022/data/day9.txt') as file:
    master_list = [line.rstrip() for line in file]

In [377]:
master_list

['D 2',
 'L 2',
 'U 1',
 'L 1',
 'U 2',
 'R 1',
 'L 1',
 'D 2',
 'U 2',
 'R 2',
 'U 1',
 'L 2',
 'R 1',
 'D 1',
 'L 2',
 'R 1',
 'U 2',
 'L 1',
 'U 1',
 'D 1',
 'R 1',
 'U 2',
 'D 2',
 'L 1',
 'D 1',
 'U 1',
 'L 1',
 'U 1',
 'D 2',
 'U 1',
 'L 2',
 'D 1',
 'L 1',
 'D 2',
 'U 1',
 'D 2',
 'R 1',
 'L 2',
 'R 1',
 'U 1',
 'R 2',
 'U 2',
 'D 2',
 'R 1',
 'D 1',
 'R 1',
 'U 2',
 'D 1',
 'U 2',
 'D 2',
 'L 1',
 'D 2',
 'R 2',
 'U 1',
 'L 1',
 'R 1',
 'U 2',
 'D 2',
 'R 1',
 'D 2',
 'U 1',
 'L 2',
 'R 1',
 'L 2',
 'D 2',
 'U 2',
 'D 2',
 'L 2',
 'U 1',
 'L 1',
 'D 2',
 'L 1',
 'R 2',
 'U 2',
 'D 1',
 'R 1',
 'U 2',
 'D 2',
 'U 1',
 'R 2',
 'L 1',
 'R 1',
 'L 1',
 'U 2',
 'D 2',
 'L 2',
 'D 1',
 'L 2',
 'U 2',
 'R 2',
 'L 1',
 'D 1',
 'L 2',
 'U 1',
 'L 2',
 'D 2',
 'L 2',
 'D 2',
 'R 2',
 'L 2',
 'R 1',
 'D 1',
 'L 2',
 'U 2',
 'L 2',
 'R 1',
 'D 1',
 'R 1',
 'D 2',
 'R 1',
 'U 2',
 'R 3',
 'U 1',
 'D 2',
 'U 3',
 'R 2',
 'U 3',
 'D 3',
 'U 1',
 'L 3',
 'R 1',
 'U 1',
 'D 2',
 'U 1',
 'D 2',


In [378]:
def get_directions(text_list):
    output_list = []
    for l in text_list:
        split_l = l.split()
        tup = (split_l[0].strip(), int(split_l[1]))
        output_list.append(tup)

    return output_list

In [379]:
directions = get_directions(master_list)
directions[0:10]

[('D', 2),
 ('L', 2),
 ('U', 1),
 ('L', 1),
 ('U', 2),
 ('R', 1),
 ('L', 1),
 ('D', 2),
 ('U', 2),
 ('R', 2)]

In [380]:
def get_pos_distance(h, t):
    x_dist = abs(h[0] - t[0])
    y_dist = abs(h[1] - t[1])
    return x_dist, y_dist

In [381]:
h = (1,1)
t = (0,0)

x, y = get_pos_distance(h, t)
print(x, y)

1 1


In [382]:
class Rope():
    def __init__(self):
        self.start = [0,0]
        self.head = [0,0]
        self.tail = [0,0]
        self.x_dist = 0
        self.y_dist = 0
        self.tail_coordinates = list()
        self.tail_positions = len(set(self.tail_coordinates))

    def move_head(self, direction, steps):
        if direction == 'R':
            self.head = [ self.head[0] + steps, self.head[1] ]
        elif direction == 'L':
            self.head = [ self.head[0] - steps, self.head[1] ]
        elif direction == 'U':
            self.head = [ self.head[0], self.head[1] + steps ]
        else:
            self.head = [ self.head[0], self.head[1] - steps]
        return

    def move_horizontal(self):
        if (self.x_dist > 1):
            # If moving to the right
            if self.head[0] > self.tail[0]:
                self.tail[0] = self.tail[0] + (self.x_dist - 1)
            # If moving to the left
            else:
                self.tail[0] = self.tail[0] - (self.x_dist - 1)
        else:
            pass

        return

    def move_vertical(self):
        if (self.y_dist > 1):
            # If moving up
            if self.head[1] > self.tail[1]:
                self.tail[1] = self.tail[1] + (self.y_dist - 1)
            # If moving down
            else:
                self.tail[1] = self.tail[1] - (self.y_dist - 1)
        else:
            pass

        return

    def snap_vertical(self):
        # Snap to same x coordinate
        self.tail[0] = self.head[0]

        # Move vertically to trailing position
        self.move_vertical()

    def snap_horizontal(self):
        # Snap to same y coordinate
        self.tail[1] = self.head[1]

        # Move horizontally to trailing position
        self.move_horizontal()

    def move_tail(self):
        # Horizontal move only
        if (self.y_dist == 0) and (abs(self.x_dist)>1):
            self.move_horizontal()
            
        # Vertical move only
        elif (self.x_dist == 0) and (abs(self.y_dist)>1):
            self.move_vertical()

        # Diagonal snap to y
        elif (abs(self.x_dist) == 1) and (abs(self.y_dist) > 1):
            self.snap_vertical()

        # Diagonal snap to x
        elif (abs(self.y_dist) == 1) and (abs(self.x_dist) > 1):
            self.snap_horizontal()

        # Diagonal jump with vertical move
        elif (abs(self.y_dist) > 1) and (abs(self.x_dist) > 1):
            self.move_vertical()
            self.move_horizontal()

        # # Diagonal jump with horizontal move
        # elif (abs(self.x_dist) > 1) and (abs(self.y_dist) > 0):
        #     self.move_horizontal()

        return

    def calc_distance(self):
        self.x_dist = abs(self.head[0] - self.tail[0])
        self.y_dist = abs(self.head[1] - self.tail[1])
        return
        
    def move_steps(self, instruction):
        # Move step by step
        for n in range(0, instruction[1]):
            # Move head
            self.move_head(instruction[0], 1)

            # Calculate distance
            self.calc_distance()

            # Move tail to follow
            self.move_tail()

            # Add tail position to set of coordinates
            self.tail_coordinates.append((self.tail[0], self.tail[1]))
            # Recalculate number of tail positions visited
            self.tail_positions = len(self.tail_coordinates)
            
        return

    def move(self, instruction):
        # Move head
        self.move_steps(instruction)

        return self.tail

    def run(self, directions):
        for dir in directions:
            self.move(dir)

    def update_position(self, head_pos):
        # Update head position
        self.head = head_pos

        # Calculate distance
        self.calc_distance()

        # Move tail to follow
        self.move_tail()

        # Add tail position to set of coordinates
        self.tail_coordinates.append((self.tail[0], self.tail[1]))
        # Recalculate number of tail positions visited
        self.tail_positions = len(set(self.tail_coordinates))

        return self.tail

In [383]:
r = Rope()
print(r.start, r.head, r.tail, r.tail_positions)

[0, 0] [0, 0] [0, 0] 0


In [384]:
r.run(directions)

In [385]:
r.tail_positions

11350

In [390]:
class LongRope():
    def __init__(self, knots):
        self.knots = [Rope() for x in range(0, knots-1)]
        self.start = [0,0]
        self.head = [0,0]
        self.tail = [0,0]
        self.tail_coordinates = []
        self.tail_positions = len(self.tail_coordinates)
        print(f"LONG ROPE CREATED WITH {len(self.knots)} KNOTS")

    def increment_rope(self, instruction):
        for n in range(0, instruction[1]):
            step_instruction = (instruction[0], 1)
            # Move first knot according to instruction
            self.knots[0].move(step_instruction)

            # Propagate position to all knots
            for n in range(1, len(self.knots)):
                self.knots[n].update_position(self.knots[n-1].tail)
                # self.show_coordinates()
                # Get position of tail node
                # tail_knot = self.knots[-1]
                # self.tail_coordinates += list(self.knots[-1].tail_coordinates)
                # self.tail_positions = len(set(self.tail_coordinates))
            # print("TAIL COORDINATES: ", self.knots[-1].tail_coordinates)
            # print("TAIL POSITION: ", self.knots[-1].tail)

            
            

        return

    def run(self, directions):
        for dir in tqdm(directions):
            # print("-----------------------")
            # print(f"INSTRUCTION {idx+1}")
            self.increment_rope(dir)
        return

    def show_coordinates(self): 
        print_statement = ""
        for n in range(0, len(self.knots)):
            print_statement += f"KNOT {n}: {self.knots[n].head}\n"
        print(print_statement)
        return

    def print_grid(self):
        row = ".....................\n"
        all_rows = []
        for n in range(0, 21):
            all_rows.append(row)

        for idx, knot in enumerate(self.knots):
            if idx == 0:
                idx = 'H'
            x_val = 11 + knot.head_coordinates
            # row[]

        full_grid = ""
        for row in all_rows:
            full_grid += row

        print(full_grid)
        return





In [391]:
long_rope = LongRope(knots=10)
long_rope.run(directions)

LONG ROPE CREATED WITH 9 KNOTS




























































































































































































































































































































100%|██████████| 2000/2000 [00:48<00:00, 40.96it/s]


In [392]:
long_rope.show_coordinates()

KNOT 0: [170, -240]
KNOT 1: [170, -241]
KNOT 2: [170, -242]
KNOT 3: [170, -243]
KNOT 4: [170, -242]
KNOT 5: [170, -241]
KNOT 6: [170, -240]
KNOT 7: [170, -239]
KNOT 8: [170, -238]



In [393]:
len(set(long_rope.knots[-1].tail_coordinates))

2597

In [350]:
long_rope.print_grid()

.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................
.....................

